<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-7/blob/main/ImagenesAntiguasApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Creación de una aplicación para la clasificación de imágenes antiguas**

Hecho por: Ignacio Jorquera Ferrat y Patricia Vallejo Fernández

________________________________________________________________________________

En este notebook se desarrollará una aplicación que utilice el modelo creado con el anterior cuaderno. 

Esta aplicación consistirá en una interfaz simple para introducir una imagen que queramos clasificar mediante un botón y un mensaje con el resultado.
________________________________________________________________________________

## 1. Instalaciones e importaciones

Lo primero que necesitamos es instalar fastai e importar las librerías necesarias.

In [10]:
%%capture
!pip install fastai --upgrade

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

También necesitaremos importar nuestro modelo conseguido con el notebook anterior desde drive.

In [ ]:
!gdown --id 1ujFDBhxjIpurUr2gRVd5M8B5-WeRCj6Y

## 2. Creación de la interfaz de la aplicación

Vamos a cargar el modelo que hemos descargado.

Después, obtenemos los widgets necesarios para nuestra aplicación.
- Botón: Un botón para subir archivos, en este caso lo utilizaremos para subir las imágenes para las que el modelo hará una predicción.
- Output: Mensaje de salida con la predicción del modelo.
- Label: Texto previo al botón.

In [ ]:
path = Path()
learn_inf = load_learner(path/'export.pkl', cpu=True)

btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

Definimos el método que detectará que hemos subido una imagen y que la enviará al modelo para que devuelva la predicción.

El valor devuelto tiene la forma de "Predicción: [clase], Probabilidad: [probabilidad]", siendo "clase" la clase o etiqueta que el modelo ha predecido y "probabilidad" la probabilidad de que sea dicha clase.

In [ ]:
def on_data_change(change):
    lbl_pred.value = ''

    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(224,244))

    pred, pred_idx, probs = learn_inf.predict(img)

    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

Añadimos al botón de subida el método anterior.

Por último, mostramos esta interfaz.
Como vemos, consta de:
- Mensaje "Select your image": Etiqueta con el mensaje previo.
- Botón de updload: Botón que utilizaremos para subir la imagen desde nuestro ordenador.
- Mensaje final: Output con la predicción y la probabilidad de dicha predicción.

In [ ]:
btn_upload.observe(on_data_change, names=['data'])

display(VBox([widgets.Label('Select your image'), btn_upload, out_pl, lbl_pred]))

Hemos probado que esta aplicación funcione correctamente subiendo varias imágenes, mostrándose esta última en el notebook.

En esta imagen, el modelo predice la clase "Iglesia" con un 87.86% de acierto.

## 3. Despliegue de la aplicación con Heroku

El último paso será desplegar esta aplicación con ayuda de Heroku. Para ello: 

1. Crearemos un fichero "Procfile" en nuestro repositorio de github con el siguiente contenido:

  web: voila --port=$PORT --no-browser --enable_nbextensions=True ImagenesAntiguasApp.ipynb

  Donde el último elemento es este notebook.


2. Crearemos un fichero "requirements.txt" en nuestro repositorio de github con el siguiente contenido:

  -f https://download.pytorch.org/whl/torch_stable.html

  torch==1.8.1+cpu

  torchvision==0.9.1+cpu

  fastai

  voila

  ipywidgets


3. Necesitamos incluir el modelo exportado en nuestro repositorio.


4. Entraremos en Heroku, haremos una cuenta y en dashboard, creamos nuestra aplicación. Se nos pedirá un nombre de aplicación y el repositorio de github.

Tras una espera, nuestra aplicación estará lista para ser utilizada. 

El link para utilizar nuestra aplicación es: https://imagenes-antiguas-app.herokuapp.com/